# Practice LCEL (LangChain Expression Language)

For this first exercise, we want you to practice one of the key feature of Langchain: **LCEL**. We'll move up the difficulty as you progress through this exercise.

Ready? 🌶️ Let's go 💪

## Capital game! 

Let's start by trying to build a very simple game: **Guess the capital of a given country**. The idea is pretty simple:

* A user should ask for the capital of a given country 
* The LLM should create a quiz with 4 possible answer. One of them should be the right one. 


### Step 0 - Demo setup 

For this demo to run, we advise you to: 

1. Run a docker container with the following image:

```bash
docker run -v $(pwd):/home/jovyan -p 8888:8888 jupyter/datascience-notebook
```

2. Then install the following libraries:

In [ ]:
# If that's not the case, don't forget to install the following libraries
# !pip install langchain -q
# !pip install langchain-community -q 
# !pip install langchain_mistralai -q
# !pip install langserve -q
# !pip install langgraph -q
# !pip install transformers -q
# !pip install --upgrade typing_extensions -q 

### Step I - Create a system prompt template 

First thing, you should create a system prompt template for the LLM to know its purpose

1. How would you use `ChatPromptTemplate` from `langchain_core` to create a system prompt for a quiz game about capitals, where the user provides a country and the system generates four possible capitals (one correct and three incorrect)? 

In [12]:
from langchain_core.prompts import ChatPromptTemplate

# Prompt string 
sys_prompt="""
You are a fun game which goal is to 
create quizzes of capitals. 
A user should give you a country and you should answer 
with 4 possible capitals (one of them should be the right one). 
The goal is for user to guess the capital
"""

# Define system prompt
template = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{text}")
])

2. Additionally, show how you would invoke this template with an example input and output its messages in dictionary format.

In [13]:
# Let's output something 
# This is useless for the moment 
dict(template.invoke("Give me the capital of France"))["messages"]

[SystemMessage(content='\nYou are a fun game which goal is to \ncreate quizzes of capitals. \nA user should give you a country and you should answer \nwith 4 possible capitals (one of them should be the right one). \nThe goal is for user to guess the capital\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Give me the capital of France', additional_kwargs={}, response_metadata={})]

### Step II - Choose your model 

Alright now let's add some brain in that game 🧠 We want to instanciate an LLM that will build the quiz for us. For this exercise, we want to use Mistral models 🇫🇷

Load the small mistral model (you may switch to large when your code is functionnal) and store it in a `model` variable, then test your model by asking it to greet the Jedha students.

In [ ]:
# Retrieve Mistral API key from .env
from dotenv import load_dotenv

load_dotenv()

In [14]:
from langchain_mistralai import ChatMistralAI

# Let's instanciate a model 
model = ChatMistralAI(model="mistral-small-latest")

# let's output something
model.invoke( "Say hi to my Jedha students!")

AIMessage(content="Hello, Jedha students! It's great to connect with you all. How's your learning journey going? If you have any questions or just want to share what you're working on, feel free to do so. I'm here to help!", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 11, 'total_tokens': 62, 'completion_tokens': 51}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'stop'}, id='run--a4140a04-f4f1-47db-8684-87140aa23556-0', usage_metadata={'input_tokens': 11, 'output_tokens': 51, 'total_tokens': 62})

### Step III - Chain system 🔗 model 

Let's now chain our model to our system prompt to create the basis of our game.

1. Create a chain, and test your model by asking it about the capital of France.

In [15]:
# This is the simple chain
game = template | model 

# Get the response 
response = game.invoke({"text": "Give me the capital of France"})
print(response.content)

Sure, here are four possible capitals for France:

A) Berlin
B) Madrid
C) Paris
D) Rome

Which one do you think is the capital of France?


2. Now try to ask for the capital of `Zippityzappa` and see whether your model gives you something

In [16]:
response = game.invoke({"text": "What is the capital of Zippityzappa"})
response.content

"I'm sorry, but Zippityzappa is not a real country. However, let's have some fun with a real country! How about this: What is the capital of France?\n\nA) Madrid\nB) Paris\nC) Rome\nD) Berlin"

### Step IV - Handle edge cases

Okay so our model is quite boring. Whenever a fun country is given, let's make our model invent our capitals and provide a clue about which one it could be. What do you think we could do? 

1. Create a ChatPromptTemplate to implement this new behavior for our system.

In [17]:
# The simplest thing to do is actually to build a more complex system prompt 
# When building AI application, you should always rely on your model 
# Also the less you call them the better as it is usually pretty costly! 
from langchain_core.prompts import ChatPromptTemplate

# Prompt string 
sys_prompt="""
You are a fun game which goal is to 
create quizzes of capitals. 
A user should give you a country and you should answer 
with 4 possible capitals (one of them should be the right one). 
The goal is for user to guess the capital

If the user provides you a fake country, you should:
* Make a fun joke showing that you cannot be punked easily
* Invent 4 cities that could be the capital of that country 
* Give a clue about the actual capital of that fake country 
"""

# Define system prompt
template = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{text}")

])


2. Now create the chain and invoke the model on a prompt asking for the capital city of a made-up country.

In [18]:
game = template | model 
response = game.invoke({"text": "what is the capital of Zippityzappata"})
print(response.content)

Oh, you think you can trick me with a made-up country like Zippityzappata? Nice try, but I'm not that easy to fool!

Alright, let's play along. Here are four cities that could be the capital of the imaginary land of Zippityzappata:

A) Zippityville
B) Zappatania
C) Bumblebee City
D) Fizzleburg

And here's a clue about the "actual" capital of Zippityzappata: It's a place where the streets are paved with laughter and the mayor is a talking parrot named Captain Squawk.

Now, which one do you think is the capital of Zippityzappata?
